In [ ]:
# Langchain/graph/smith
## - https://python.langchain.com/v0.1/docs/integrations/chat/groq/
## - https://python.langchain.com/docs/tutorials/agents/

In [22]:
# !pip install langchain-groq
# !pip install langchain-anthropic
# !pip install langchain-community
# !pip install langgraph
# %pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

In [ ]:
# Tavily Api key

In [14]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [ ]:
# groq Api Key

In [19]:
groq_api_key = getpass.getpass(prompt='Groq Api Key')

Groq Api Key ········


In [23]:
# langchain api key

In [24]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [20]:
# Create the agent
memory = MemorySaver()
model = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you. San Francisco is a great city with a lot to offer. If you have any questions about San Francisco or anything else, feel free to ask!\n\n(No tools were used to generate this response.)", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 1215, 'total_tokens': 1268, 'completion_time': 0.082913784, 'prompt_time': 0.073527673, 'queue_time': 0.017733869, 'total_time': 0.156441457}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb04ea28-2579-48e0-ba5e-437555770c42-0', usage_metadata={'input_tokens': 1215, 'output_tokens': 53, 'total_tokens': 1268})]}}
----
{'agent': {'messages': [AIMessage(content="The weather in San Francisco today is typically mild, with temperatures ranging from the mid 50s to the mid 60s Fahrenheit. It's currently a bit cloudy, but it should clear up later in the day. 

In [52]:
# define tools
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1735383146, 'localtime': '2024-12-28 02:52'}, 'current': {'last_updated_epoch': 1735382700, 'last_updated': '2024-12-28 02:45', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 187, 'wind_dir': 'S', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 89, 'cloud': 100, 'feelslike_c': 13.6, 'feelslike_f': 56.5, 'windchill_c': 10.5, 'windchill_f': 50.9, 'heatindex_c': 11.6, 'heatindex_f': 53.0, 'dewpoint_c': 11.2, 'dewpoint_f': 52.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 7.6, 'gust_kph': 12.2}}"}, {'url': 'https://www.peoplesweather.com/weather/

In [53]:
from langchain_core.messages import HumanMessage
model = ChatGroq(groq_api_key=groq_api_key,model="llama3-8b-8192")
response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [54]:
model_with_tools = model.bind_tools(tools)
model_with_tools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x12399a1d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x12352aa10>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [55]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Hi'}, 'id': 'call_q8me', 'type': 'tool_call'}]


In [ ]:
response = model_with_tools.invoke([HumanMessage(content="weather")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Provide weather information for Bangalore, ensuring all required parameters for the weather API are correct.")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
# Create agent

In [58]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [59]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='73471ef4-748b-44bb-a1ef-b9209ed6f983'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3q2m', 'function': {'arguments': '{"query":"hi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 941, 'total_tokens': 1018, 'completion_time': 0.064166667, 'prompt_time': 0.1310648, 'queue_time': 0.021447349000000004, 'total_time': 0.195231467}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fda70539-c0c6-4663-847b-7d4c8803ac5b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'hi'}, 'id': 'call_3q2m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 941, 'output_tokens': 77, 'total_tokens': 1018}),
 ToolMessage(content='[{"url": "https://dictionary.cambridge.org/dictionary/english/hi", "content": "HI 

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

In [63]:
# adding memory
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [64]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [65]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 1906, 'total_tokens': 1910, 'completion_time': 0.003333333, 'prompt_time': 0.235239738, 'queue_time': -0.29129517, 'total_time': 0.238573071}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-a9da6623-a701-4750-8cc7-6e3ef52191eb-0', usage_metadata={'input_tokens': 1906, 'output_tokens': 4, 'total_tokens': 1910})]}}
----


In [66]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 960, 'total_tokens': 964, 'completion_time': 0.003333333, 'prompt_time': 0.119509664, 'queue_time': 0.020652443000000006, 'total_time': 0.122842997}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-59858a53-3fc0-4d04-8749-2bd45872ca47-0', usage_metadata={'input_tokens': 960, 'output_tokens': 4, 'total_tokens': 964})]}}
----


In [67]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I'm not sure what your name is.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 1909, 'total_tokens': 1919, 'completion_time': 0.008333333, 'prompt_time': 0.34269442, 'queue_time': -0.407184249, 'total_time': 0.351027753}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-68504310-c7ed-4d89-99b3-c7b8779c6f5b-0', usage_metadata={'input_tokens': 1909, 'output_tokens': 10, 'total_tokens': 1919})]}}
----
